### Actividad 5: Taller SQL (SQL Server)
Para completar el taller recuerda estas sentencias de SQL:

DML (modifica datos):

* INSERT: agrega filas a una tabla.
* UPDATE: modifica columnas de filas existentes.
* DELETE: elimina filas.

SELECT (consulta datos):

* SELECT: elige columnas a devolver (puede incluir expresiones).
* WHERE: filtra filas (antes de agrupar).
* GROUP BY: agrupa filas para usar agregados (SUM, COUNT, AVG, MIN, MAX).
* HAVING: filtra grupos (después de GROUP BY).
* ORDER BY: ordena el resultado.
* TOP (N): limita la cantidad de filas devueltas (útil con ORDER BY).

In [70]:
import pyodbc
import pandas as pd

server = 'cdsc-bit.database.windows.net'
database = 'BDPruebasBIT'
username = 'KENAYD2016'
password = 'Bit202507'
# Instalar dependencias y repositorio de Microsoft (no me servia el comando sudo)

available = pyodbc.drivers()
driver_name = None
for candidate in ('ODBC Driver 18 for SQL Server', 'ODBC Driver 17 for SQL Server'):
	if any(candidate in d for d in available):
		driver_name = candidate
		break

if driver_name is None:
	for d in available:
		if 'SQL Server' in d or 'ODBC' in d:
			driver_name = d
			break

if not driver_name:
	raise RuntimeError(
		"No suitable ODBC driver found. Available drivers: "
		f"{available}. Install Microsoft ODBC Driver for SQL Server (see https://learn.microsoft.com/sql/connect/odbc/download-odbc-driver)."
	)

driver = f'{{{driver_name}}}'

conn_str = (
	f"DRIVER={driver};"
	f"SERVER={server};"
	f"DATABASE={database};"
	f"UID={username};"
	f"PWD={password};"
	"Encrypt=yes;"
	"TrustServerCertificate=no;"
	"Connection Timeout=30;"
)

try:
	conn = pyodbc.connect(conn_str)
	cur = conn.cursor()
except pyodbc.Error as e:
	print("Connection failed:", e)
	print("Tried connection string:", conn_str)
	print("Available ODBC drivers on this system:", available)
	raise

### Ejercicios

### 1. Lista ProductID, Name, ListPrice de SalesLT.Product
(SELECT)

In [71]:
respuesta1 = pd.read_sql_query("""
SELECT ProductID, Name, ListPrice from SalesLT.Product
""", conn)
respuesta1

C:\Users\User\AppData\Local\Temp\ipykernel_11788\3039303763.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  respuesta1 = pd.read_sql_query("""


,ProductID,Name,ListPrice
0,680,"HL Road Frame - Black, 58",1431.50
1,706,"HL Road Frame - Red, 58",1431.50
2,707,"Sport-100 Helmet, Red",34.99
3,708,"Sport-100 Helmet, Black",34.99
4,709,"Mountain Bike Socks, M",9.50
...,...,...,...
290,995,ML Bottom Bracket,101.24
291,996,HL Bottom Bracket,121.49
292,997,"Road-750 Black, 44",539.99
293,998,"Road-750 Black, 48",539.99


### 2. Devuelve productos con ListPrice > 1000 mostrando ProductNumber, Name, ListPrice.
(Recuerda usar WHERE)

In [72]:
respuesta2 = pd.read_sql_query("SELECT ProductNumber, Name, ListPrice from SalesLT.Product where ListPrice > 1000", conn)
respuesta2

C:\Users\User\AppData\Local\Temp\ipykernel_11788\3199062220.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  respuesta2 = pd.read_sql_query("SELECT ProductNumber, Name, ListPrice from SalesLT.Product where ListPrice > 1000", conn)


,ProductNumber,Name,ListPrice
0,FR-R92B-58,"HL Road Frame - Black, 58",1431.50
1,FR-R92R-58,"HL Road Frame - Red, 58",1431.50
2,FR-R92R-62,"HL Road Frame - Red, 62",1431.50
3,FR-R92R-44,"HL Road Frame - Red, 44",1431.50
4,FR-R92R-48,"HL Road Frame - Red, 48",1431.50
...,...,...,...
81,BK-T44U-54,"Touring-2000 Blue, 54",1214.85
82,BK-R79Y-40,"Road-350-W Yellow, 40",1700.99
83,BK-R79Y-42,"Road-350-W Yellow, 42",1700.99
84,BK-R79Y-44,"Road-350-W Yellow, 44",1700.99


### 3. Muestra el TOP 5 productos más caros con ProductID, Name, ListPrice.

In [73]:
respuesta3 = pd.read_sql_query("""
SELECT TOP 5 ProductID, Name, ListPrice FROM SalesLT.Product
ORDER BY ListPrice DESC
""", conn)
respuesta3

C:\Users\User\AppData\Local\Temp\ipykernel_11788\1329627953.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  respuesta3 = pd.read_sql_query("""


,ProductID,Name,ListPrice
0,749,"Road-150 Red, 62",3578.27
1,750,"Road-150 Red, 44",3578.27
2,751,"Road-150 Red, 48",3578.27
3,752,"Road-150 Red, 52",3578.27
4,753,"Road-150 Red, 56",3578.27


### 4. Lista ProductID, p.Name AS ProductName, c.Name AS CategoryName para productos con categoría
(INNER JOIN).

In [74]:
respuesta4 = pd.read_sql_query("""
SELECT p.ProductID,
    p.Name AS ProductName,
    c.Name AS CategoryName
FROM SalesLT.Product p
INNER JOIN SalesLT.ProductCategory c
  ON p.ProductCategoryID = c.ProductCategoryID    
""", conn)
respuesta4

C:\Users\User\AppData\Local\Temp\ipykernel_11788\1552749354.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  respuesta4 = pd.read_sql_query("""


,ProductID,ProductName,CategoryName
0,771,"Mountain-100 Silver, 38",Mountain Bikes
1,772,"Mountain-100 Silver, 42",Mountain Bikes
2,773,"Mountain-100 Silver, 44",Mountain Bikes
3,774,"Mountain-100 Silver, 48",Mountain Bikes
4,775,"Mountain-100 Black, 38",Mountain Bikes
...,...,...,...
290,930,HL Mountain Tire,Tires and Tubes
291,931,LL Road Tire,Tires and Tubes
292,932,ML Road Tire,Tires and Tubes
293,933,HL Road Tire,Tires and Tubes


### 5. Obtén pedidos del año 2008 en adelante (o el año que aplique en tu base) con SalesOrderID, OrderDate, CustomerID, TotalDue y el nombre del cliente (FirstName + ' ' + LastName).
(JOIN + WHERE)

In [75]:
respuesta5 = pd.read_sql_query("""
    SELECT soh.SalesOrderID,
           soh.OrderDate,
           soh.CustomerID,
           soh.TotalDue,
           (c.FirstName + ' ' + c.LastName) AS CustomerName
    FROM SalesLT.SalesOrderHeader soh
    INNER JOIN SalesLT.Customer c
      ON soh.CustomerID = c.CustomerID
    WHERE YEAR(soh.OrderDate) >= 2008
""", conn)
respuesta5

C:\Users\User\AppData\Local\Temp\ipykernel_11788\3208473795.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  respuesta5 = pd.read_sql_query("""


,SalesOrderID,OrderDate,CustomerID,TotalDue,CustomerName
0,71782,2008-06-01,29485,43962.7901,Catherine Abel
1,71935,2008-06-01,29531,7330.8972,Cory Booth
2,71938,2008-06-01,29546,98138.2131,Christopher Beck
3,71899,2008-06-01,29568,2669.3183,Donald Blanton
4,71895,2008-06-01,29584,272.6468,Walter Brian
5,71885,2008-06-01,29612,608.1766,Richard Byham
6,71915,2008-06-01,29638,2361.6403,Rosmarie Carroll
7,71867,2008-06-01,29644,1170.5376,Brigid Cavendish
8,71858,2008-06-01,29653,15275.1977,Pei Chow
9,71796,2008-06-01,29660,63686.2708,Anthony Chor


### 6. Para cada ProductID, suma la cantidad vendida (SUM(OrderQty)). Devuelve solo productos con ventas totales > 1000, ordenados de manera descendente.
Usa GROUP BY, HAVING y ORDER BY

In [76]:
pd.read_sql_query("""
    SELECT TOP 10
        ProductID,
        SUM(OrderQty) as TotalQuantitySold
    FROM SalesLT.SalesOrderDetail
    GROUP BY ProductID
    ORDER BY TotalQuantitySold DESC
""", conn)

#Esto fue pa chequiar el having

C:\Users\User\AppData\Local\Temp\ipykernel_11788\2445957527.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  pd.read_sql_query("""


,ProductID,TotalQuantitySold
0,864,87
1,884,57
2,877,55
3,870,54
4,712,52
5,715,51
6,708,51
7,875,51
8,883,51
9,880,50


In [77]:
respuesta6 = pd.read_sql_query("""
    SELECT 
        ProductID,
        SUM(OrderQty) as TotalQuantitySold
    FROM SalesLT.SalesOrderDetail
    GROUP BY ProductID
    HAVING SUM(OrderQty) > 1000
    ORDER BY TotalQuantitySold DESC
""", conn)
respuesta6

#no devuelve nada

C:\Users\User\AppData\Local\Temp\ipykernel_11788\4160352752.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  respuesta6 = pd.read_sql_query("""


,ProductID,TotalQuantitySold


### 7. Productos sin ventas: lista productos que no aparecen en SalesLT.SalesOrderDetail.
Usa LEFT JOIN + WHERE

In [78]:
respuesta7 = pd.read_sql_query("""
    SELECT 
        p.ProductID,
        p.Name,
        p.ListPrice
    FROM SalesLT.Product p
    LEFT JOIN SalesLT.SalesOrderDetail sod 
        ON p.ProductID = sod.ProductID
    WHERE sod.ProductID IS NULL
""", conn)
respuesta7

C:\Users\User\AppData\Local\Temp\ipykernel_11788\477834641.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  respuesta7 = pd.read_sql_query("""


,ProductID,Name,ListPrice
0,680,"HL Road Frame - Black, 58",1431.50
1,706,"HL Road Frame - Red, 58",1431.50
2,709,"Mountain Bike Socks, M",9.50
3,710,"Mountain Bike Socks, L",9.50
4,713,"Long-Sleeve Logo Jersey, S",49.99
...,...,...,...
148,942,"ML Mountain Frame-W - Silver, 38",364.09
149,943,"LL Mountain Frame - Black, 40",249.79
150,946,LL Touring Handlebars,46.09
151,950,ML Crankset,256.49


### 8. Clientes por país: total de clientes por CountryRegion (usa Address + CustomerAddress).
Podrias necesitar: COUNT(DISTINCT CustomerID).

In [79]:
respuesta8 = pd.read_sql_query("""
    SELECT a.CountryRegion,
           COUNT(DISTINCT ca.CustomerID) AS TotalCustomers
    FROM SalesLT.Address a
    INNER JOIN SalesLT.CustomerAddress ca
        ON a.AddressID = ca.AddressID
    GROUP BY a.CountryRegion
    ORDER BY TotalCustomers DESC
""", conn)
respuesta8

C:\Users\User\AppData\Local\Temp\ipykernel_11788\2365360157.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  respuesta8 = pd.read_sql_query("""


,CountryRegion,TotalCustomers
0,United States,263
1,Canada,106
2,United Kingdom,38


### 9. Ticket promedio por año: AVG(TotalDue) por YEAR(OrderDate).

In [80]:
respuesta9 = pd.read_sql_query("""
    SELECT YEAR(OrderDate) AS Year,
           AVG(TotalDue) AS AverageTicket
    FROM SalesLT.SalesOrderHeader
    GROUP BY YEAR(OrderDate)
    ORDER BY Year
""", conn)
respuesta9

C:\Users\User\AppData\Local\Temp\ipykernel_11788\3442264263.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  respuesta9 = pd.read_sql_query("""


,Year,AverageTicket
0,2008,29884.4873
